In [31]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('lab4').getOrCreate()
df = spark.read.csv('DS_2019_public.csv', inferSchema=True, header=True)

In [32]:
from typing import Literal
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

@udf(IntegerType())
def climate_code_to_binary(code: Literal[1, 2, 3, 4, 5]) -> Literal[0, 1]:
    match code:
        case 3:
            return 0
        case 1 | 2 | 4 | 5:
            return 1

In [33]:
df = df.withColumn(
    'Class',
    climate_code_to_binary(df['Climate_Region_Pub'])
).select(
    'Class',
    'DOLELCOL',
    'KWHCOL',
    'DOLELOTH',
)
df.show()

+-----+--------+--------+--------+
|Class|DOLELCOL|  KWHCOL|DOLELOTH|
+-----+--------+--------+--------+
|    1|  16.793| 181.998| 337.315|
|    1|  48.901| 184.459| 296.353|
|    1| 101.048|1063.022| 512.731|
|    1|     0.0|     0.0| 619.862|
|    1|  45.132|  274.53| 853.549|
|    1|  253.21|1224.714|1762.032|
|    0|  700.88|6129.163|1128.303|
|    1|  54.482| 401.146| 348.027|
|    1|  94.694|1080.527| 433.394|
|    1|     0.0|     0.0| 194.909|
|    1| 245.173|1343.896| 1489.74|
|    0|     0.0|     0.0| 1491.27|
|    1| 100.871| 889.995|  316.25|
|    1|  47.321| 557.754| 198.509|
|    1| 244.207|2396.059| 732.486|
|    1|  57.573| 441.678| 176.533|
|    1|     0.0|     0.0| 237.423|
|    1|     0.0|     0.0| 525.915|
|    1|  89.792| 785.259|1133.852|
|    1| 720.897|3936.861|1305.659|
+-----+--------+--------+--------+
only showing top 20 rows



In [34]:
df_train, df_test = df.randomSplit([0.75, 0.25])
print(f'Records for training: {df_train.count()}')
print(f'Records for testing: {df_test.count()}')

Records for training: 8118
Records for testing: 2757


In [35]:
from pyspark.ml.classification import RandomForestClassifier

rf_model = RandomForestClassifier(
    labelCol='Class',
    numTrees=50,
).fit(df_train)

IllegalArgumentException: features does not exist. Available: Class, DOLELCOL, KWHCOL, DOLELOTH